In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# import codecs
# df = codecs.open('../wellcome.csv', "rb", encoding='latin-1') 

In [3]:
df = pd.read_csv("../wellcome.csv", encoding ='latin-1')

In [4]:
# test
# dec = "ejfwohie"
# dec.encode('utf-8')

# doesn't work: df.apply(lambda x: str(x).encode('utf-8')) 

# works: df['Journal title'].str.encode('utf-8');

Inconsistencies: 
- the PubMed ID if a PMC ID 
- Publisher name  in different ways.  For example, OUP, Oxford University Press, and O.U.P.
- Journal name. For example, PNAS, Proceedings of the National Academy of Sciences, P.N.A.S.
- different prices cited for the same journal.  
    - When these differences are small, it can be assumed that this reflects fluctuations in currency prices.  
    - Where they are significant, then it should be assumed that other funders have contributed to the cost of that APC, and the figure cited is the Wellcome contribution to that APC.

In [5]:
df['Journal title'].describe()

count         2126
unique         984
top       PLoS One
freq            92
Name: Journal title, dtype: object

In [6]:
# convert to lower case
df['Journal title'] = df['Journal title'].str.lower()

In [7]:
# convert & to and
df['Journal title'] = df['Journal title'].str.replace('&', 'and')

In [8]:
# convert J, Jnl, Jounral, Jounal, Journals to journal
regex_journal = re.compile('j(([a-z])*)?')
df['Journal title'] = df['Journal title'].str.replace(regex_journal, 'journal')

# df['Journal title'].replace(to_replace='j(([a-z])*)?', value='journal', inplace=True, regex=True)

In [9]:
# delete the
df['Journal title'] = df['Journal title'].str.replace('the', '')

In [10]:
# remove sections of a journal following colon, comma, dash
regex_colon = re.compile(':  ?.*')
regex_comma = re.compile(',  ?.*')
regex_dash = re.compile('-  ?.*')
df['Journal title'] = df['Journal title'].str.replace(regex_colon, '')
df['Journal title'] = df['Journal title'].str.replace(regex_comma, '')
df['Journal title'] = df['Journal title'].str.replace(regex_dash, '')

In [11]:
# Sci. to Science
df['Journal title'].replace(to_replace='sci.?([a-z].*)?', value='science', inplace=True, regex=True)
# np.sort(df['Journal title'].unique())
# journals = df['Journal title'].unique()
# journals.sort()

In [12]:
# Biol to Biological
df['Journal title'].replace(to_replace='(biol )', value='biological ', inplace=True, regex=True)
df['Journal title'].replace(to_replace='(biol)$', value='biological', inplace=True, regex=True)

array(['psychological medicine', 'biomacromolecules', 'journal med chem',
       'journal org chem', 'journal of medicinal chemistry',
       'journal of proteome research', 'mol pharm', 'acs chemical biology',
       'journal of chemical information and modeling', 'biochemistry',
       'gastroenterology', 'journal of biological chemistry',
       'journal of immunology', 'acs chemical neuroscience', 'acs nano',
       'american chemical society', 'analytical chemistry',
       'bioconjournal chemistry', 'journal of medicinal chemistry ',
       'journal of  american chemical society', 'chest',
       'journal of neurophysiology', 'journal of physiology',
       ' journal of neurophysiology', 'american journal of psychiatry',
       'americal journal of psychiatry', 'behavioral neuroscience',
       'emotion', 'health psychology', 'journal of abnormal psychology',
       'journal of consulting and clinical psychology',
       'journal of experimental psychology',
       'journal of fa

In [13]:
# space at the end of the sentence
df['Journal title'] = df['Journal title'].str.strip()

In [14]:
# Regex replace: 
    # biochimica et bioohysica acta.* to bba
df['Journal title'].replace(to_replace='.*acta.*', value='bba', inplace=True, regex=True)

# Fromatting replace: 
    # '  ' to ' '
df['Journal title'].replace(to_replace='  ', value=' ', inplace=True, regex=False)

In [15]:
# Replace of 1:1 cases:
    # Epidemiol = Epidemiology
    # US English vs British English: Behavioural and Behavioral
    # BJOPHTHALMOL = British Journal of Ophthalmology
    # Opthalmology = Ophthalmology
    # Sex Transm Infect. = Sexually Transmitted Infections 
    # 'American Journal for Clinical Nutrition' = 'American Journal of Clinical Nutrition'
    # Am to American
    # molecluar to molecular

# Regex replace: 
    # Proceedings of the National Academy of Sciences of.* to PNAS
    # Chem to Chemistry
    # Int, Intl to International

In [16]:
# test
df['Journal title'].unique()

array(['psychological medicine', 'biomacromolecules', 'journal med chem',
       'journal org chem', 'journal of medicinal chemistry',
       'journal of proteome research', 'mol pharm', 'acs chemical biology',
       'journal of chemical information and modeling', 'biochemistry',
       'gastroenterology', 'journal of biological chemistry',
       'journal of immunology', 'acs chemical neuroscience', 'acs nano',
       'american chemical society', 'analytical chemistry',
       'bioconjournal chemistry', 'journal of  american chemical society',
       'chest', 'journal of neurophysiology', 'journal of physiology',
       'american journal of psychiatry', 'americal journal of psychiatry',
       'behavioral neuroscience', 'emotion', 'health psychology',
       'journal of abnormal psychology',
       'journal of consulting and clinical psychology',
       'journal of experimental psychology',
       'journal of family psychology', 'psychological assessment',
       'psychological revie

## Five most common journals and the total articles for each

In [18]:
df_agg = df.groupby(['Journal title']).agg({'count':sum})
res = df_agg.apply(lambda x: x.order(ascending=False).head(5))

AttributeError: 'DataFrame' object has no attribute 'order'

## Mean, median, and standard deviation of the open-access cost per article for each journal 

## Open access prices paid by subject area ?